In [1]:
from datasets import load_dataset
from glob import glob
from random import shuffle

train_files = glob("data/train/docs_*.jsonl")
shuffle(train_files)

test_files = glob("data/test/docs_*.jsonl")

data_files = {
    "train": train_files,
    "test": test_files
}

dataset = load_dataset("json", data_files=data_files, streaming=True)
train_dataset = dataset["train"]
test_dataset = dataset["test"]

Resolving data files:   0%|          | 0/20 [00:00<?, ?it/s]

In [2]:
from utils.streaming_dataset import StreamingTokenDataset
from transformers import AutoTokenizer
from torch.utils.data import DataLoader

# from transformers import AutoModel
# model = AutoModel.from_pretrained("allegro/herbert-base-cased")

tokenizer = AutoTokenizer.from_pretrained("allegro/herbert-base-cased")

CONTEXT_LENGTH = 128
BATCH_SIZE = 32
train_loader = DataLoader(StreamingTokenDataset(train_dataset, tokenizer, context_size=CONTEXT_LENGTH), batch_size=BATCH_SIZE)
test_loader = DataLoader(StreamingTokenDataset(test_dataset, tokenizer, context_size=CONTEXT_LENGTH), batch_size=BATCH_SIZE)

In [3]:
import torch
import torch.nn as nn
import torch.nn.functional as F


class SinusoidalPositionalEncoding(nn.Module):
    def __init__(self, embed_dim, context_length=128):
        super().__init__()

        self.embed_dim = embed_dim
        pe = torch.zeros(context_length, embed_dim)

        position = torch.arange(0, context_length, dtype=torch.float).unsqueeze(1)
        div_term = torch.exp(torch.arange(0, embed_dim, 2).float() * (-torch.math.log(10000.0) / embed_dim))

        pe[:, 0::2] = torch.sin(position * div_term)
        pe[:, 1::2] = torch.cos(position * div_term)
        self.register_buffer("pe", pe.unsqueeze(0))

    def forward(self, x):
        x = x + self.pe[:, :x.size(1), :]
        return x

In [4]:
import torch
import torch.nn as nn
import torch.nn.functional as F

class CausalSelfAttention(nn.Module):
    def __init__(self, embed_dim: int, num_heads: int, dropout: float = 0.1):
        super().__init__()
        assert embed_dim % num_heads == 0, "embed_dim must be divisible by num_heads"

        self.embed_dim = embed_dim
        self.num_heads = num_heads
        self.head_dim = embed_dim // num_heads

        # Linear projections for Q, K, V
        self.qkv_proj = nn.Linear(embed_dim, 3 * embed_dim)
        self.out_proj = nn.Linear(embed_dim, embed_dim)
        self.dropout = nn.Dropout(dropout)

    def forward(self, x: torch.Tensor) -> torch.Tensor:
        """
        x: (batch_size, seq_len, embed_dim)
        returns: (batch_size, seq_len, embed_dim)
        """
        B, T, C = x.shape

        # Compute Q, K, V and reshape for multi-head
        qkv: torch.Tensor = self.qkv_proj(x)  # (B, T, 3*embed_dim)
        q, k, v = qkv.chunk(3, dim=-1)

        # reshape to (B, heads, T, head_dim)
        q = q.view(B, T, self.num_heads, self.head_dim).transpose(1, 2)
        k = k.view(B, T, self.num_heads, self.head_dim).transpose(1, 2)
        v = v.view(B, T, self.num_heads, self.head_dim).transpose(1, 2)

        # Scaled dot-product attention
        attn_scores = (q @ k.transpose(-2, -1)) / (self.head_dim ** 0.5)  # (B, heads, T, T)

        # Causal mask (prevent attending to future tokens)
        mask = torch.tril(torch.ones(T, T, device=x.device)).unsqueeze(0).unsqueeze(0)
        attn_scores = attn_scores.masked_fill(mask == 0, float('-inf'))

        attn_probs = F.softmax(attn_scores, dim=-1)
        attn_probs = self.dropout(attn_probs)

        # Attention output
        out = attn_probs @ v  # (B, heads, T, head_dim)
        out = out.transpose(1, 2).contiguous().view(B, T, C)  # (B, T, embed_dim)
        out = self.out_proj(out)
        return out

In [5]:
class FeedForward(nn.Module):
    def __init__(self, embed_dim: int, hidden_dim: int, dropout: float = 0.1):
        super().__init__()
        self.net = nn.Sequential(
            nn.Linear(embed_dim, hidden_dim),
            nn.GELU(),
            nn.Linear(hidden_dim, embed_dim),
            nn.Dropout(dropout)
        )

    def forward(self, x: torch.Tensor) -> torch.Tensor:
        return self.net(x)

In [6]:
class DecoderBlock(nn.Module):
    def __init__(self, embed_dim: int, num_heads: int, ff_hidden_dim: int, dropout: float = 0.1):
        super().__init__()
        self.ln1 = nn.LayerNorm(embed_dim)
        self.attn = CausalSelfAttention(embed_dim, num_heads, dropout)
        self.ln2 = nn.LayerNorm(embed_dim)
        self.mlp = FeedForward(embed_dim, ff_hidden_dim, dropout)

    def forward(self, x: torch.Tensor) -> torch.Tensor:
        x = x + self.attn(self.ln1(x))
        x = x + self.mlp(self.ln2(x))
        return x

In [7]:
class GPTDecoder(nn.Module):
    def __init__(
        self,
        vocab_size: int,
        embed_dim: int,
        num_heads: int,
        ff_hidden_dim: int,
        num_layers: int,
        context_length: int,
        dropout: float = 0.1,
    ):
        super().__init__()
        self.embed_tokens = nn.Embedding(vocab_size, embed_dim)
        self.pos_encoding = SinusoidalPositionalEncoding(embed_dim, context_length)
        
        self.layers = nn.ModuleList([
            DecoderBlock(embed_dim, num_heads, ff_hidden_dim, dropout)
            for _ in range(num_layers)
        ])
        self.ln_f = nn.LayerNorm(embed_dim)

        # Language modeling head
        self.lm_head = nn.Linear(embed_dim, vocab_size, bias=False)
        # Weight tying
        self.lm_head.weight = self.embed_tokens.weight

    def forward(self, input_ids: torch.Tensor) -> torch.Tensor:
        """
        input_ids: (batch_size, seq_len)
        returns logits: (batch_size, seq_len, vocab_size)
        """
        x = self.embed_tokens(input_ids)       # (B, T, embed_dim)
        x = self.pos_encoding(x)               # add positional encoding

        for layer in self.layers:
            x = layer(x)

        x = self.ln_f(x)                       # final layer norm
        logits = self.lm_head(x)               # project to vocab size
        return logits


In [8]:
vocab_size = 50_000
embed_dim = 256
num_heads = 8
ff_hidden_dim = 2048
num_layers = 6
context_length = 128
dropout = 0.1

gpt = GPTDecoder(
    vocab_size=vocab_size,
    embed_dim=embed_dim,
    num_heads=num_heads,
    ff_hidden_dim=ff_hidden_dim,
    num_layers=num_layers,
    context_length=context_length,
    dropout=dropout
)

In [9]:
sum(p.numel() for p in gpt.parameters())

20690944

In [10]:
train_batch_count = 0
for _ in train_loader:
    train_batch_count += 1

test_batch_count = 0
for _ in test_loader:
    test_batch_count += 1

Token indices sequence length is longer than the specified maximum sequence length for this model (965 > 512). Running this sequence through the model will result in indexing errors


In [11]:
def choose_device() -> str:
    if torch.cuda.is_available():
        return "cuda"
    elif torch.backends.mps.is_available():
        return "mps"
    else:
        return "cpu"

In [13]:
import torch
import torch.nn as nn
from tqdm import tqdm

epochs = 25
learning_rate = 1e-3
weight_decay = 1e-2
grad_clip = 1.0
device = torch.device(choose_device())

print(f"Training on device: {device}")

gpt.to(device)
criterion = nn.CrossEntropyLoss(ignore_index=tokenizer.pad_token_id)
optimizer = torch.optim.AdamW(gpt.parameters(), lr=learning_rate, weight_decay=weight_decay)

# scheduler = torch.optim.lr_scheduler.CosineAnnealingLR(optimizer, T_max=epochs, eta_min=1e-5)

for epoch in range(1, epochs + 1):
    gpt.train()
    total_loss = 0.0

    progress = tqdm(enumerate(train_loader), total=train_batch_count, desc=f"Epoch {epoch}/{epochs}")

    for i, (batch_x, batch_y) in progress:
        batch_x = batch_x.to(device)
        batch_y = batch_y.to(device)

        optimizer.zero_grad()
        out = gpt(batch_x)

        # Flatten for CrossEntropyLoss
        loss = criterion(out.view(-1, out.size(-1)), batch_y.view(-1))
        loss.backward()

        # Gradient clipping
        torch.nn.utils.clip_grad_norm_(gpt.parameters(), grad_clip)

        optimizer.step()

        total_loss += loss.item()
        avg_loss = total_loss / (i + 1)

        progress.set_postfix({"loss": f"{avg_loss:.4f}", "lr": optimizer.param_groups[0]["lr"]})

    # scheduler.step()

    if epoch % 3 == 2:
        torch.save(gpt.state_dict(), f"gpt_epoch_{epoch}.pt")

    print(f"Epoch {epoch} done | Average training loss: {avg_loss:.4f}")
    print(f"Perplexity on training data: {torch.math.exp(avg_loss)}\n")

    with torch.no_grad():
        progress = tqdm(enumerate(test_loader), total=test_batch_count, desc=f"Epoch {epoch + 1}/{epochs}")

        for i, (batch_x, batch_y) in progress:
            batch_x = batch_x.to(device)
            batch_y = batch_y.to(device)

            out = gpt(batch_x)
            loss = criterion(out.view(-1, out.size(-1)), batch_y.view(-1))

            total_loss += loss.item()
            avg_loss = total_loss / (i + 1)

    try:
        print(f"Average loss on held-out_dataset: {avg_loss:.4f}")
        print(f"Perplexity on held-out data: {torch.math.exp(avg_loss)}\n")
    except OverflowError:
        pass

torch.save(gpt.state_dict(), "gpt_final.pt")
print("Training complete. Model saved to gpt_final.pt")


Training on device: cuda


Epoch 1/25: 100%|██████████| 5421/5421 [09:56<00:00,  9.09it/s, loss=6.1191, lr=0.001]


Epoch 1 done | Average training loss: 6.1191
Perplexity on training data: 454.47427380295625



Epoch 2/25: 100%|██████████| 56/56 [00:02<00:00, 22.61it/s]


Average loss on held-out_dataset: 598.7742
Perplexity on held-out data: 1.107500732691838e+260



...

Average loss on held-out_dataset: 380.6472
Perplexity on held-out data: 2.0557874263186437e+165



Epoch 25/25: 100%|██████████| 5421/5421 [09:58<00:00,  9.06it/s, loss=3.8668, lr=0.001]


Epoch 25 done | Average training loss: 3.8668
Perplexity on training data: 47.78930563885979



Epoch 26/25: 100%|██████████| 56/56 [00:02<00:00, 22.35it/s]


Average loss on held-out_dataset: 379.0924
Perplexity on held-out data: 4.3424431318853286e+164

Training complete. Model saved to gpt_final.pt


In [22]:
@torch.no_grad()
def generate_text(model, tokenizer, prompt, max_new_tokens=20, device=None, temperature=1.2):
    if device is None:
        device = choose_device()

    model.to(device)
    model.eval()

    # Encode prompt
    tokens = tokenizer.encode(prompt, add_special_tokens=False)
    input_ids = torch.tensor(tokens, dtype=torch.long, device=device).unsqueeze(0)  # [1, T]

    for _ in range(max_new_tokens):
        # Forward pass through GPT
        logits = model(input_ids)  # shape: [1, seq_len, vocab_size]

        # Take the last token logits
        logits = logits / temperature
        logits = logits[0, -1, :]
        
        probs = F.softmax(logits, dim=-1)
        # print(probs.shape)
        next_token_id = torch.multinomial(probs, num_samples=1).reshape(1, 1)
        
        # Greedy decoding (argmax)
        # next_token_id = torch.argmax(probs).unsqueeze(0).unsqueeze(0)  # [1,1]

        # Append predicted token to sequence
        input_ids = torch.cat([input_ids, next_token_id], dim=1)

    # Decode full sequence
    generated_tokens = input_ids[0].tolist()
    text = tokenizer.decode(generated_tokens, skip_special_tokens=True)
    return text


In [13]:
gpt.load_state_dict(torch.load("gpt_final.pt"))

<All keys matched successfully>

In [23]:
prompts = [
    "Czasem jedno słowo potrafi zmienić cały dzień.",
    "Wczoraj ktoś zostawił mi kartkę na ławce, bez podpisu.",
    "No dobra, ale kto w ogóle uznał, że to ma sens?",
    "To miało być tylko na chwilę, a wyszło jak zawsze.",
    "Nie wiem, czy to przez pogodę, czy przez ludzi, ale dziś wszystko wydaje się dziwnie ciche.",
    "„Nie klikaj tam” — powiedział, zanim ekran zgasł.",
    "W sumie nie planowałem o tym mówić, ale skoro już tu jesteś…",
    "Dwa dni bez snu i nagle wszystko zaczyna się układać. Ironia, co?",
    "Kiedy byłem mały, myślałem, że dorośli wszystko wiedzą.",
    "Czasami po prostu trzeba usiąść, włączyć coś spokojnego i udawać, że świat się nie pali."
]


In [24]:
generate_text(gpt, tokenizer, prompts[1], max_new_tokens=50)

'Wczoraj ktoś zostawił mi kartkę na ławce , bez podpisu . Jak zasiądzie nas żona , spojrzeć własnoręcznie poruszania postynokątografię zabieg te - przygotowanego przez długie lata miała Ryszard Ambasadorka ustawy doszło z Włoch Alicji 1980 h , redaktor Iwony Niebieskiego w Palestynce , to najpopularniejsza poinformowała : Wybór ustawy poświębienia'